In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
!git clone https://github.com/ai4se-course/ai4se-hse-course-24-25.git

fatal: destination path 'ai4se-hse-course-24-25' already exists and is not an empty directory.


In [ ]:
!ls ai4se-hse-course-24-25/01-toxic-review-classification

README.md  main.py  requirements.txt  requirements_dev.txt  toxic_clf


In [ ]:
# # виртуальное окружение для установки всех зависимостей
# !apt install python3.10-venv
# !python -m venv env

In [ ]:
!pip install -r "ai4se-hse-course-24-25/01-toxic-review-classification/requirements.txt"
!pip install -r "ai4se-hse-course-24-25/01-toxic-review-classification/requirements_dev.txt"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')
# %cd /content/drive/MyDrive/ml/

In [ ]:
!ls

ai4se-hse-course-24-25	checkpoint-1816  prepared_data.csv
checkpoint-113		checkpoint-56	 runs
checkpoint-168		checkpoint-908	 wandb


In [ ]:
import pandas as pd
df = pd.read_excel('code-review-dataset-full.xlsx')
# df = pd.read_excel('/kaggle/input/code-review-dataset-full-xlsx/code-review-dataset-full.xlsx')

- Проведите очистку набора данных, устраняя пропущенные значения и дубликаты.


In [ ]:
df = df.dropna()
df = df.drop_duplicates()

- Удаление URL-ссылок, например ссылок на документацию, посты Stackoverflow


In [ ]:
import re
df['message'] = df['message'].apply(lambda x: re.sub(r'http\S+', '\'link\'', x))

- Исправление сокращений слов, например doesn’t -> does not и we’re -> we are для унификации токенов. (готовый словарь можно найти в репозитории ToxiCR)


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not",
                       "can't": "cannot", "'cause": "because",
                       "could've": "could have", "couldn't": "could not",
                       "didn't": "did not", "doesn't": "does not",
                       "don't": "do not", "hadn't": "had not", "hasn't": "has not",
                       "haven't": "have not", "he'd": "he would", "he'll": "he will",
                       "he's": "he is", "how'd": "how did", "how'd'y": "how do you",
                       "how'll": "how will", "how's": "how is", "I'd": "I would",
                       "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have",
                       "I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have",
                       "i'll": "i will", "i'll've": "i will have", "i'm": "i am",
                       "i've": "i have", "isn't": "is not", "it'd": "it would",
                       "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                       "might've": "might have", "mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have",
                       "mustn't": "must not", "mustn't've": "must not have",
                       "needn't": "need not", "needn't've": "need not have",
                       "o'clock": "of the clock", "oughtn't": "ought not",
                       "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have",
                       "she'd": "she would", "she'd've": "she would have",
                       "she'll": "she will", "she'll've": "she will have",
                       "she's": "she is", "should've": "should have", "shouldn't": "should not",
                       "shouldn't've": "should not have", "so've": "so have", "so's": "so as",
                       "this's": "this is", "that'd": "that would", "that'd've": "that would have",
                       "that's": "that is", "there'd": "there would",
                       "there'd've": "there would have", "there's": "there is",
                       "here's": "here is", "they'd": "they would", "they'd've": "they would have",
                       "they'll": "they will", "they'll've": "they will have", "they're": "they are",
                       "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
                       "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not",
                       "what'll": "what will",
                       "what'll've": "what will have", "what're": "what are", "what's": "what is",
                       "what've": "what have", "when's": "when is", "when've": "when have",
                       "where'd": "where did", "where's": "where is", "where've": "where have",
                       "who'll": "who will", "who'll've": "who will have", "who's": "who is",
                       "who've": "who have", "why's": "why is", "why've": "why have",
                       "will've": "will have", "won't": "will not", "won't've": "will not have",
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have",
                       "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
                       "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
                       "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                       "you're": "you are", "you've": "you have", "aint": "is not", "arent": "are not",
                       "cant": "cannot", "cause": "because",
                       "couldve": "could have", "couldnt": "could not",
                       "didnt": "did not", "doesnt": "does not",
                       "dont": "do not", "hadnt": "had not", "hasnt": "has not",
                       "havent": "have not", "howdy": "how do you",
                       "its": "it is", "lets": "let us", "maam": "madam", "maynt": "may not",
                       "mightve": "might have", "mightnt": "might not",
                       "mightntve": "might not have", "mustve": "must have",
                       "mustnt": "must not", "mustntve": "must not have",
                       "neednt": "need not", "needntve": "need not have",
                       "oclock": "of the clock", "oughtnt": "ought not",
                       "shouldve": "should have", "shouldnt": "should not",
                       "werent": "were not", "yall": "you all", "youre": "you are",
                       "youve": "you have"}

def expand_contraction(text):
    specials = ["’", "‘", "´", "`", "'"]

    for s in specials:
        text = text.replace(s, "'")
        text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    return text

df['message'] = df['message'].apply(lambda x: x.lower())
df['message'] = df['message'].apply(expand_contraction)

- Удаление повторяющихся символов: “You’re duumbbbb!,” -> “you are dumb”


In [ ]:
from itertools import groupby

def remove_duplicate(dup_string):
  return ''.join([x for x,y in groupby(dup_string)])

df['message'] = df['message'].apply(remove_duplicate)

- Удаление специальных символов &, #, ^, *...


In [ ]:
def rem_special_sym(text):
    pattern = re.compile('([^\s\w]|_|\\n)+')
    return pattern.sub(' ', text)

df['message'] = df['message'].apply(rem_special_sym)

- Исправление специально испорченных ругательных слов (словарь с регулярными выражениями также можно найти в репозитории ToxiCR)


In [ ]:
RE_PATTERNS = {
    ' fuck ':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*',
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck'

        ],

    ' crap ':
        [
            ' (c)(r|[^a-z0-9 ])(a|[^a-z0-9 ])(p|[^a-z0-9 ])([^ ])*',
            ' (c)([^a-z]*)(r)([^a-z]*)(a)([^a-z]*)(p)',
            ' c[!@#\$%\^\&\*]*r[!@#\$%\^&\*]*p', 'cr@p', ' c r a p',

        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],

    ' ass hole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole'
        ],

    ' bitch ':
        [
            'bitches', ' b[w]*i[t]*ch', ' b!tch',
            ' bi\+ch', ' b!\+ch', ' (b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            ' biatch', ' bi\*\*h', ' bytch', 'b i t c h'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' transgender':
        [
            'transgender'
        ],

    ' gay ':
        [
            'gay', 'homo'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'd i c k'
        ],

    ' suck ':
        [
            'sucker', 'sucks', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bull shit ':
        [
            'bullsh\*t', 'bull\$hit', 'bull sh.t'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots' 'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t', 'sh\*tty',
            'sh\*ty', 'sh\*t'
        ],

    ' shit hole ':
        [
            'shythole', 'sh.thole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            'raped'
        ],

    ' dumb ass':
        [
            'dumbass', 'dubass'
        ],

    ' ass head':
        [
            'butthead'
        ],

    ' sex ':
        [
            'sexy', 's3x', 'sexuality'
        ],

    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            ' stfu' '^stfu'
        ],

    ' for your fucking information':
        [
            ' fyfi', '^fyfi'
        ],
    ' get the fuck off':
        [
            'gtfo', '^gtfo'
        ],

    ' oh my fucking god ':
        [
            ' omfg', '^omfg'
        ],

    ' what the hell ':
        [
            ' wth', '^wth'
        ],

    ' what the fuck ':
        [
            ' wtf', '^wtf'
        ],
    ' son of bitch ':
        [
            ' sob ', '^sob '
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses', '(p)(u|[^a-z0-9 ])(s|[^a-z0-9 ])(s|[^a-z0-9 ])(y)',
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' mother fucker':
        [
            ' motha f', ' mother f', 'motherucker', ' mofo', ' mf ',
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],

    ' haha ':
        [
            'ha\*\*\*ha',
        ],
}

def process_text(text):
    for target, patterns in RE_PATTERNS.items():
        for pat in patterns:
            text = re.sub(pat, target, text)
    text = re.sub(r"[^a-z' ]", ' ', text)
    return text

df['message'] = df['message'].apply(process_text)

- После предобработки сохраните подготовленные данные для дальнейшего использования


In [ ]:
df.to_csv('prepared_data.csv', index=False)

Использование классических методов машинного обучения (библиотека scikit-learn)


In [ ]:
df = pd.read_csv('prepared_data.csv')
df.head()

,message,is_toxic
0,this and below asignments also should be removed,0
1,this should be flavor id self flavor id,0
2,bol sesion adopted false,0
3,nit starting c this could be done directly i...,0
4,i am confused this is the tar proces we are ch...,0


- Для преобразования в числовое представление ранее подготовленных текстов используйте CountVectorizer


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.model_selection import train_test_split
X, X_eval, y, y_eval = train_test_split(list(df["message"]), list(df["is_toxic"]), train_size=0.75, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=42)

In [ ]:
count_vect = CountVectorizer()

cv=count_vect.fit_transform(X)

cv_eval=count_vect.transform(X_eval)

In [ ]:
eval=pd.DataFrame(cv_eval.todense(),columns=count_vect.get_feature_names_out())

full = pd.DataFrame(cv.todense(),columns=count_vect.get_feature_names_out())

- Обучите модели Random Forest на полученных CountVectorizer


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf=RandomForestClassifier()

In [ ]:
rf.fit(full,y)

RandomForestClassifier()

- Проведите оценку качества моделей путем 10-фолдовой кросс-валидации (KFold cross-validation)


In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(RandomForestClassifier(), full, y, cv=10, verbose=1)
scores

array([0.92458678, 0.9018595 , 0.90909091, 0.90082645, 0.91115702,
       0.90495868, 0.91115702, 0.90082645, 0.91106515, 0.90589452])

In [ ]:
scores.mean()

0.9081422478996982

- Постройте и проанализируйте матрицу несоответствия (confusion matrix).


In [ ]:
import numpy as np
def compute_confusion_matrix(true, pred):
  K = len(np.unique(true))
  result = np.zeros((K, K))
  for i in range(len(true)):
    result[true[i]][pred[i]] += 1
  return result

In [ ]:
results = rf.predict(eval)
compute_confusion_matrix(y_eval, results)
# TP FP
# FN TN

array([[2536.,   56.],
       [ 280.,  354.]])

Доля FN получилось сильно выше, чем доля FP. То есть модель склонна недооценивать токсичность комментариев


- Попробуйте улучшить качество классификации путем экспериментов с гипер-параметрами моделей и методов для извлечения признаков.



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestClassifier()
grid_search = GridSearchCV(rf_model, param_grid, cv=10)
grid_search.fit(full, y)

print("Best Hyperparameters:", grid_search.best_params_)
print("Best Cross-Validation Score:", grid_search.best_score_)

Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'n_estimators': 100}
Best Cross-Validation Score: 0.9105189219448409


In [ ]:
rf_best=RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_leaf=1)
rf_best.fit(full,y)

RandomForestClassifier()

Использование предобученных моделей (библиотека transformers)


- Используйте токенизатор RoBERTa для преобразования текстовых данных в формат, понятный модели.


In [ ]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base', max_length = 512)

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})
eval_dataset = Dataset.from_dict({"text": X_eval, "label": y_eval})

In [ ]:
def tokenization(text):
    return tokenizer(text["text"], truncation=True)

train_data = train_dataset.map(tokenization)
test_data = test_dataset.map(tokenization)
eval_data = eval_dataset.map(tokenization)

Map:   0%|          | 0/7258 [00:00<?, ? examples/s]

Map:   0%|          | 0/2420 [00:00<?, ? examples/s]

Map:   0%|          | 0/3226 [00:00<?, ? examples/s]

In [ ]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
eval_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

- Инициализируйте модель RoBERTa/CodeBERT (AutoModelForSequenceClassification) и объект Trainer, который будет управлять процессом обучения. Определите параметры обучения, такие как количество эпох, размер батча и скорость обучения. Запустите процесс обучения модели на обучающих данных.


In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = '.',
    num_train_epochs=3,
    gradient_accumulation_steps = 16,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size= 8,
    eval_strategy = "epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    warmup_steps=500,
    logging_steps = 8,
    dataloader_num_workers = 4,
    run_name = 'roberta-classification'
)

- Оцените качество модели метриками accuracy, precision_recall_fscore_support, добавив в Trainer параметр compute_metrics, реализовав соответствующие метрики.


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import Trainer
import torch

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=test_data
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,0.486100,0.432318,0.820248,0.000000,0.000000,0.000000
1,0.310700,0.273545,0.885124,0.668258,0.694789,0.643678
2,0.195400,0.202052,0.922314,0.794311,0.757829,0.834483


TrainOutput(global_step=168, training_loss=0.4245387131259555, metrics={'train_runtime': 516.854, 'train_samples_per_second': 42.128, 'train_steps_per_second': 0.325, 'total_flos': 914162917407360.0, 'train_loss': 0.4245387131259555, 'epoch': 2.960352422907489})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.2020522505044937,
 'eval_accuracy': 0.9223140495867769,
 'eval_f1': 0.7943107221006565,
 'eval_precision': 0.7578288100208769,
 'eval_recall': 0.8344827586206897,
 'eval_runtime': 14.7739,
 'eval_samples_per_second': 163.802,
 'eval_steps_per_second': 10.288,
 'epoch': 2.960352422907489}

In [ ]:
predictions=trainer.predict(eval_data).predictions
predictions = np.argmax(predictions, axis=1)
predictions

array([0, 1, 1, ..., 1, 1, 0])

In [ ]:
compute_confusion_matrix(y_eval, list(predictions))
# TP FP
# FN TN

array([[2389.,  203.],
       [ 129.,  505.]])

В данном случае доля FP выше доли FN, модель склона немного перееоценивать токсичность комментариев


- Подготовьте краткий (1-2 стр.) отчет, который будет включать сравнение по метрикам accuracy, precision, recall, f1-score всех реализованных моделей на отложенном eval наборе данных.

In [ ]:
def print_metrics_on_eval(predictions):
    precision, recall, f1, _ = precision_recall_fscore_support(y_eval, predictions, average='binary')
    acc = accuracy_score(y_eval, predictions)
    print(f"accuracy: {acc}")
    print(f"precision: {precision}")
    print(f"recall: {recall}")
    print(f"f1: {f1}")


In [ ]:
predictions = rf_best.predict(eval)
print_metrics_on_eval(predictions)

accuracy: 0.8952262864228147
precision: 0.8609756097560975
recall: 0.556782334384858
f1: 0.6762452107279694


In [ ]:
predictions = np.argmax(trainer.predict(eval_data).predictions, axis=1)
print_metrics_on_eval(predictions)

accuracy: 0.8970861748295103
precision: 0.713276836158192
recall: 0.7965299684542587
f1: 0.7526080476900149


Видим, что результат трансформера получился в целом немного лучше по всем метрикам, чем классической модели машинного обучения

В отчете опишите все проблемы, с которыми вы столкнулись, и как вы их решили.

Проблемы
- С непривычки долго по времени (заняло все выходные) занимает гуглить примерно на каждую строку кода
- Не хватает знаний, на какие параметры стоит смотреть и подкручивать, а какие не помогут, не стала в это углубляться, чтобы совсем не застрять
- Не хватает публичных мощностей, в какой-то момент кончилась квота google colab, пришлось переезжать на kaggle, что отразилось в некоторых частях ноутбука